<a href="https://colab.research.google.com/github/liupengzhouyi/LearningPytorch/blob/master/20200310/ConvolutionNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution network

## import package

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## setting device

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## setting super parameters

In [0]:
input_size = 28 * 28        # 784
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

## download data

In [4]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


## create data set

In [0]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Create net

In [0]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## Instantiation net

> used GPU

In [0]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

## create loss function

In [0]:
criterion = nn.CrossEntropyLoss()

## Create optimizer

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

## Train model

In [10]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.2743
Epoch [1/2], Step [200/600], Loss: 0.2805
Epoch [1/2], Step [300/600], Loss: 0.1765
Epoch [1/2], Step [400/600], Loss: 0.1859
Epoch [1/2], Step [500/600], Loss: 0.2125
Epoch [1/2], Step [600/600], Loss: 0.1474
Epoch [2/2], Step [100/600], Loss: 0.1847
Epoch [2/2], Step [200/600], Loss: 0.1104
Epoch [2/2], Step [300/600], Loss: 0.1074
Epoch [2/2], Step [400/600], Loss: 0.0778
Epoch [2/2], Step [500/600], Loss: 0.1645
Epoch [2/2], Step [600/600], Loss: 0.0433


## Test the model

In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.13 %


## Save model

In [0]:
torch.save(model.state_dict(), 'model.ckpt')

over!